# Introduction to Quantum Computing with Amazon Braket

In this workshop, we hope to introduce you to the exciting field of quantum computing using Julia and AWS quantum computing service. We will cover:

- Basic construction of quantum circuits (Bell pairs, superdense coding)
- Hybrid quantum algorithms (QAOA)
- Quantum machine learning

This workshop can be completed *entirely* with the Amazon Braket local simulator, which is free to use. You *do not* need an AWS account to participate! If you would like to run circuits on Amazon Braket's managed simulators (SV1, DM1, and TN1) or use a real quantum device, you can sign up for an AWS account, for which you will need a credit card, and you may be charged depending on your usage.

## What is Amazon Braket?

Amazon Braket is a fully managed quantum computing service that helps researchers and developers get started with the technology to accelerate research and discovery. Amazon Braket provides a development environment for you to explore and build quantum algorithms, test them on quantum circuit simulators, and run them on different quantum hardware technologies. You can read more about Amazon Braket [here](https://aws.amazon.com/braket/).

Amazon Braket provides access to a variety of quantum circuit simulators and hardware providers.


Some of these simulators (SV1, TN1, and DM1) are *managed devices* -- they run on AWS-managed infrastructure and software -- and one of them (the local simulator, which we'll use during this workshop) runs on your computer. In order to use the local simulator, you'll need to follow the [pre-workshop installation instructions](README.md).

## Why not use Yao?

Many of you are likely familiar with the wonderful [Yao.jl](https://github.com/QuantumBFS/Yao.jl) package. Our goal with this workshop is to give a very "bottom up" introduction to quantum computing. The syntax we'll use is different from that of Yao.jl and we will build up a training procedure for our QML approach directly using Flux, for example. If you like the Yao syntax best -- use Yao! If you want to use Yao's great set of templates for quantum algorithms -- please do! Our goal here is to show how some of these templates arise and how to build them ourselves. (In fact, the Yao dev team is [planning](https://github.com/QuantumBFS/AWSBraket.jl) to support Amazon Braket as a backend, so you may soon be able to use both.)

### Preliminaries: Package Imports

In [1]:
using Pkg
Pkg.add("PyCall")
Pkg.build("PyCall")
Pkg.add("AWS")
using PyCall

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/299304989a5e6473d985212c28928899c74e9421/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/169bb8ea6b1b143c5cf57df6d34d022a7b60c6db/build.log`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
# Braket imports
awsbraket = pyimport("braket.aws");
devices   = pyimport("braket.devices");
circuit   = pyimport("braket.circuits");

# we're going to hijack Python's printing
pyimport("sys")."stdout" = PyTextIO(stdout)
pyimport("sys")."stderr" = PyTextIO(stderr)

PyObject <PyIO object at 0x16720fe80>

## Choosing a device

Throughout this tutorial we will be using the Amazon Braket local simulator. This runs entirely on a device of your choosing -- your laptop, your desktop, a Jupyter notebook running in the cloud, or an EC2 instance -- and is completely free. You will not need an AWS account to use the local simulator. It is a "state vector" simulator, which means that the amount of memory it needs scales exponentially in the number of qubits. This is because a state vector simulator stores a full representation of the quantum state in memory. For this reason, we're only going to run relatively small circuits (<=18 qubits). This has the happy side effect of allowing our simulations to finish quickly. If you want to run simulations with more qubits, one option is to use the managed simulators available on AWS. To do so, you'll need to create an AWS account and onboard with Braket. You can read more about Amazon Braket's various simulators and QPUs [here](https://docs.aws.amazon.com/braket/). Note that when you onboard to Braket you'll receive a free hour of simulator time per month for the first 12 months.

In [3]:
device = devices.LocalSimulator()

PyObject <braket.devices.local_simulator.LocalSimulator object at 0x166f4ee80>

## Simple circuits: building them and making measurements

First, we'll build a simple [Bell pair](https://en.wikipedia.org/wiki/Bell_state) and make measurements on it. This circuit maximally entangles two qubits using two gates: the [Hadamard](https://en.wikipedia.org/wiki/Hadamard_transform#Quantum_computing_applications) and controlled-NOT. The Hadamard puts the first qubit into a superposition of $|0\rangle$ and $|1\rangle$ states, and the `CNOT` (equivalent to `CX`) gate flips the second qubit if and only if the first one is in the $|1\rangle$ state.

**Note**: Because the Braket SDK is written in Python, the first qubit is qubit 0! 

In [4]:
bell = circuit.Circuit().h(0).cnot(0, 1)
task = device.run(bell, shots=100)
@show task.result().measurement_counts

(task.result()).measurement_counts = Dict{Any, Any}("00" => 49, "11" => 51)


Dict{Any, Any} with 2 entries:
  "00" => 49
  "11" => 51

In [5]:
py"print"(bell)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


We can also use the Braket SDK to make measurements. The SDK supports two types of measurements: `shots=0` (exact) and `shots > 0` (sampling). When we make a sampling measurement with `n` shots, we simulate the results of running the circuit `n` times on a real quantum device. When we make an "exact" measurement, we simulate what the result should converge to after $n \to \inf$. Thus, `measurement_counts` is only supported for `shots>0` computations. But we can ask the simulator to compute `expectation`s (the mean value of measuring an observable $\hat{O}$) and `variance`s in both `shots=0` and `shots>0` computations.

In the example below, we'll compute the expectation value of the `X` gate on both qubits. Note that we'll perform a `shots>0` measurement, so what the simulator will actually do is compute 100 output states and then generate an expectation value from those results.

In [6]:
x = circuit.Observable.X()
bell.expectation(py"$x @ $x", target=[0,1])
task = device.run(bell, shots=100)
@show task.result().values[1]

(task.result()).values[1] = 1.0


1.0

Similarly, we can compute the expectation value for `shots=0` and see that (up to floating point noise) the results match. This is expected for such a small circuit, but for larger and more complex circuits the two may not be equal. Generally as the number of shots increases we expect expectation values to approach their exact (`shots=0`) values.

In [7]:
x = circuit.Observable.X()
bell.expectation(py"$x @ $x", target=[0,1])
task = device.run(bell, shots=0)
@show task.result().values[1]

(task.result()).values[1] = 0.9999999999999996


0.9999999999999996

We can also compute variances:

In [8]:
bell = circuit.Circuit().h(0).cnot(0, 1)
z = circuit.Observable.Z()
bell.expectation(z, target=[0])
bell.variance(z, target=[0])
task = device.run(bell, shots=0)
@show task.result().values[1]
@show task.result().values[2]

(task.result()).values[1] = 0.0
(task.result()).values[2] = 0.9999999999999998


0.9999999999999998

We can also measure *amplitudes* with the Braket SDK. After applying the circuit, we end up with $ \hat{C}|0...0\rangle = |\psi\rangle$. The *amplitude* measures $\langle c | \psi \rangle$ for some bitstring $c$. The square of the amplitude is the probability of observing state $|c\rangle$ when we make a final measurement.

In [9]:
bell = circuit.Circuit().h(0).cnot(0, 1)
bell.amplitude(state=["00", "01", "10", "11"])
task = device.run(bell, shots=0)
@show task.result().values[1]

(task.result()).values[1] = Dict{Any, Any}("00" => 0.7071067811865475 + 0.0im, "10" => 0.0 + 0.0im, "11" => 0.7071067811865475 + 0.0im, "01" => 0.0 + 0.0im)


Dict{Any, Any} with 4 entries:
  "00" => 0.707107+0.0im
  "10" => 0.0+0.0im
  "11" => 0.707107+0.0im
  "01" => 0.0+0.0im

### Suggestions for further exploration

- Try modifying our Bell circuit with different gates. What happens if you replace `CNOT` with `CY`? Does it change the expectation values? What about the amplitude(s)?
- Can you think of a way to extend this procedure of constructing states which have two possible measurement outcomes -- all `0` or all `1` -- to more qubits? (These states are called "cat states".) If you get stuck, try searching online for "GHZ" circuits.

## Superdense Coding

Quantum computers can encode information in fewer bits than are necessary classically for the purpose of sending messages. This is called "superdense coding" and we'll demonstrate how it works below. We go through this example to show a (simple) demonstration of how we can use quantum circuits to accomplish real tasks.

In [10]:
circ = circuit.Circuit()
circ.h([0])
circ.cnot(0,1);

Now, by applying gates on only the first qubit (Alice's), we can pass a message to Bob.

In [11]:
message = Dict("00"=>circuit.Circuit().i(0),
               "01"=>circuit.Circuit().x(0),
               "10"=>circuit.Circuit().z(0),
               "11"=>circuit.Circuit().x(0).z(0)
          )

Dict{String, PyObject} with 4 entries:
  "00" => PyObject Circuit('instructions': [Instruction('operator': I('qubit_co…
  "10" => PyObject Circuit('instructions': [Instruction('operator': Z('qubit_co…
  "11" => PyObject Circuit('instructions': [Instruction('operator': X('qubit_co…
  "01" => PyObject Circuit('instructions': [Instruction('operator': X('qubit_co…

In [12]:
# Let's add a message
our_message = "11"
circ.add_circuit(message[our_message])
# now Bob disentangles the qubits to read the message
circ.cnot(0,1)
circ.h([0])
print(circ)

PyObject Circuit('instructions': [Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(0), Qubit(1)])), Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': Z('qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(0), Qubit(1)])), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]))])

In [13]:
task = device.run(circ, shots=1000)
task.result().measurement_counts

Dict{Any, Any} with 1 entry:
  "11" => 1000

## Dense random circuits

Now let's do something a little more sophisticated a build a dense circuit using layers of random gates. These circuits can be used in testing "quantum advantage". We'll also see how to apply angled gates and visualize a more interesting circuit.

In [14]:
using Random
Random.seed!(42)

function generate_random_circ(n_qubits::Int, depth::Int)
    function random_single_qubit_layer()
        return [circuit.Instruction(rand([circuit.Gate.Rx(rand()), circuit.Gate.Ry(rand()), circuit.Gate.Rz(rand())]), i)
                for i in 0:n_qubits-1]
    end
    function even_two_qubit_layer()
        return [circuit.Instruction(circuit.Gate.CNot(), [i, i+1]) for i in 1:2:n_qubits-2]
    end
    function odd_two_qubit_layer()
        return [circuit.Instruction(circuit.Gate.CNot(), [i, i+1]) for i in 0:2:n_qubits-1]
    end

    layer_fxns = [random_single_qubit_layer, even_two_qubit_layer, random_single_qubit_layer, odd_two_qubit_layer]
    circ = circuit.Circuit()
    for (layer, layer_fx) in zip(1:depth, Iterators.cycle(layer_fxns))
        for gate in layer_fx()
            circ.add_instruction(gate)
        end
    end
    return circ
end

generate_random_circ (generic function with 1 method)

We can now generate and visualize a variety of these circuits. 

In [15]:
for n_qubits in 4:4:12, n_layers in 5:5:20
    @show n_qubits, n_layers
    circ = generate_random_circ(n_qubits, n_layers)
    states = [prod(rand(["0","1"], n_qubits)) for ii in 1:10]
    circ.amplitude(state=states)
    py"print"(circ)
    task = device.run(circ, shots=0)
    println(task.result().values[1])
end

(n_qubits, n_layers) = (4, 5)
T  : |    0    |    1    |    2    |3|     4     |
                                                  
q0 : -Rx(0.533)-Ry(0.602)-----------C-Rx(0.969)---
                                    |             
q1 : -Rz(0.974)-C---------Rz(0.745)-X-Ry(0.764)---
                |                                 
q2 : -Rx(0.304)-X---------Ry(0.503)-C-Ry(0.00481)-
                                    |             
q3 : -Ry(0.937)-Rz(0.497)-----------X-Rx(0.115)---

T  : |    0    |    1    |    2    |3|     4     |

Additional result types: Amplitude(1111,0000,1011,0111,0011,1010,1000,0101,1100,0000)
Dict{Any, Any}("0101" => 0.0373088264619158 - 0.08145458721127508im, "1000" => -0.2664898091262317 - 0.07872637584485723im, "1011" => -0.07985411363084562 + 0.02351518193335491im, "0011" => 0.009350845627065448 - 0.20927099553439213im, "0000" => 0.40531591982402193 - 0.5698059849929892im, "1010" => -0.03976243330050812 - 0.0051991599159302935im, "1100" => -0.18304049853

Additional result types: Amplitude(11010000,01101111,10111011,01100001,10101111,00001010,10111111,00110010,01100110,01001001)
Dict{Any, Any}("10101111" => -0.01365116870228738 - 0.012522172845097709im, "11010000" => -0.000933680765140563 + 0.02339385245504997im, "01101111" => -0.020303343925332083 + 0.02342434453960179im, "10111011" => 0.03709637635716637 + 0.017632009111276423im, "00001010" => -0.029984188683948462 - 0.138037648539239im, "01100110" => -0.02613739081635729 + 0.0964257984020162im, "01100001" => 0.03016757944179959 + 0.04810084391319486im, "10111111" => 0.041610268109884954 + 0.0009828833270040915im, "00110010" => -0.04967039709781762 + 0.05009118791271449im, "01001001" => -0.012551618992228347 - 0.010006765513331961im)
(n_qubits, n_layers) = (12, 5)
T   : |     0     |    1    |    2     |3|    4     |
                                                     
q0  : -Rz(0.907)---Rx(0.203)------------C-Rx(0.0228)-
                                        |            
q1  : -R

Additional result types: Amplitude(001110011110,011111010110,100110011110,011111101111,110010001111,110101100110,000110000111,011010111001,010100100111,111101000001)
Dict{Any, Any}("110101100110" => -0.02742593500752173 + 0.013607534829265345im, "011111101111" => 0.008399057537081176 + 0.00612715268107509im, "011111010110" => -0.007157638123007695 + 0.008845207194955651im, "110010001111" => -0.0024190050623582506 + 0.005558769848747761im, "011010111001" => 0.005615452939862786 + 0.00029803987389070725im, "010100100111" => 0.007772884700595126 - 0.012134933746940937im, "100110011110" => -0.01159804157435809 + 0.014073278503040873im, "000110000111" => 0.005190397905243801 + 0.002183038274994741im, "111101000001" => -0.0026227661625356728 + 0.020305446214543374im, "001110011110" => 0.024840824745563532 + 0.00329836492250304im)


In a "real" simulation we would compare these output amplitudes with the observed frequencies of output states generated by running the circuits on our quantum device. In this way we could determine how noisy the quantum device is and probe things like gate fidelities and decoherence times.

### Suggestions for further exploration

- Try some of the other 2-qubit gates Braket supports in these circuits -- like `SWAP`, `iSWAP`, `CX`, `CY`, `CZ`, and others. Do they affect the runtimes and outputs of your circuit?
- Try implementing some of the examples in Braket's [getting started](https://github.com/aws/amazon-braket-examples/tree/main/examples/getting_started) or [advanced circuits algorithms](https://github.com/aws/amazon-braket-examples/tree/main/examples/advanced_circuits_algorithms) set of tutorials in Julia.